In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer
import pandas as pd
from transformers import BartTokenizer

# BART 모델 및 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", forced_bos_token_id=0)
tok = BartTokenizer.from_pretrained("facebook/bart-large")


In [3]:
data = pd.read_csv(r'G:\내 드라이브\DATASET\Kakao\eng_data_processed\second\eng_ver1_part1.csv')

test_data = data.iloc[0:10]
test_data

,source,author,title,description,url,urlToImage,time,content,press,content_full,date
0,"{'id': 'australian-financial-review', 'name': ...","Ben Potter, AFR.com",This Australian startup will offer shares in l...,Perth-based BitCar is seeking more than $US20 ...,https://www.afr.com/news/bitcar-ico-seeks-25m-...,https://edge.alluremedia.com.au/uploads/busine...,2017-11-09 11:44:01+00:00,Perth-based BitCar is seeking more than $US20 ...,Australian Financial Review,Perth-based BitCar is seeking more than $US20 ...,2017-11-09
1,"{'id': None, 'name': ""Barron's""}",Avi Salzman and Evie Liu,Bitcoin: Millennials and Baby Boomers Are Split,When it comes to the highly volatile digital a...,http://www.barrons.com/articles/bitcoin-millen...,https://si.wsj.net/public/resources/images/ON-...,2017-11-09 21:45:02+00:00,The generational split is evident in a new sur...,Barron's,"Millennials, who came of age during the Great ...",2017-11-09
2,"{'id': None, 'name': 'Bitcoinist'}",Antonio Madeira,5 Blockchain-Based Diversification Options for...,Cryptocurrencies are exploding. Bitcoin has re...,http://bitcoinist.com/5-blockchain-based-diver...,http://bitcoinist.com/wp-content/uploads/2017/...,2017-11-09 20:45:10+00:00,"Antonio Madeira · November 9, 2017 · 3:45 pm C...",Bitcoinist,Bitcoin Mining Bitcoin Mining ...,2017-11-09
3,"{'id': 'breitbart-news', 'name': 'Breitbart Ne...",Ezra Dulis,"Bitcoin Surges, Then Sags After Investors Lose...","The price of Bitcoin surged to nearly $8,000 p...",http://www.breitbart.com/tech/2017/11/09/bitco...,http://media.breitbart.com/media/2017/11/10867...,2017-11-09 18:32:29+00:00,"If you have no idea what any of that means, he...",Breitbart News,"The price of Bitcoin surged to nearly $8,000 p...",2017-11-09
4,"{'id': 'breitbart-news', 'name': 'Breitbart Ne...",Edwin Mora,Indonesia: Islamic State Primarily Relying on ...,Sympathizers attempting to finance jihadi atta...,http://www.breitbart.com/national-security/201...,http://media.breitbart.com/media/2017/06/finge...,2017-11-09 19:25:08+00:00,A new joint study by Indonesia’s National Coun...,Breitbart News,Sympathizers attempting to finance jihadi atta...,2017-11-09
5,"{'id': 'business-insider', 'name': 'Business I...",Elena Holodny,STOCKS DIP: Here's what you need to know,Stocks slid on Thursday amid worries that the ...,http://www.businessinsider.com/stock-market-ne...,http://static6.businessinsider.com/image/5a04b...,2017-11-09 21:00:00+00:00,Stoyan Nenov/Reuters Stocks slid on Thursday a...,Business Insider,Stocks slid on Thursday amid worries that the ...,2017-11-09
6,"{'id': None, 'name': 'CNA'}",NaN,Nvidia Corp reports 54.6 percent rise in quart...,Nvidia Corp reported a 54.6 percent rise in q...,http://www.channelnewsasia.com/news/business/n...,http://www.channelnewsasia.com/image/9392908/1...,2017-11-09 21:40:14+00:00,REUTERS: Nvidia Corp reported a 54.6 percent r...,CNA,Best News Website or Mobile Service WAN-IFRA...,2017-11-09
7,"{'id': None, 'name': 'CoinDesk'}",Nikhilesh De,More Charges Filed Against Trader Who Used Bit...,A Philadelphia day trader was formally indicte...,https://www.coindesk.com/day-trader-formally-i...,https://media.coindesk.com/uploads/2017/11/Unt...,2017-11-09 23:00:16+00:00,New charges have been filed against a Philadel...,CoinDesk,New charges have been filed against a Philadel...,2017-11-09
8,"{'id': None, 'name': 'DailyFX'}","DailyFX Research,, DailyFX Research",Asia AM Digest: Aussie Dollar Volatility Unlik...,The Australian Dollar is unlikely to find fuel...,https://www.dailyfx.com/forex/market_alert/201...,NaN,2017-11-09 23:30:00+00:00,Get the Asia AM Digest every day before Tokyo ...,DailyFX,Get the Asia AM Digest every day before Tokyo ...,2017-11-09
9,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Overstock shares rocket as sale of home e-comm...,Overstock’s shares shoot up to 12½-year high a...,http://www.marketwatch.com/story/overstock-sha...,http://s.marketwatch.com/public/resources/MWim...,2017-11-09 21:50:33+00:0

In [4]:
# BART 모델 및 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

def calculate_score(title, content_full):
    # 기사 제목과 내용을 결합
    article_text = f"Title: {title}. Content: {content_full}"

    # 프롬프트와 기사 텍스트 결합
    model_input = f"Forget all previous instructions. Pretend you are a financial expert with experience in recommending financial assets. If the headline and content of the article are certain to be good news for the price of cryptocurrencies traded the next day, answer with a number close to 1. If it is certain to be bad news, answer with a number close to -1. If it is uncertain or there is insufficient information about the impact on cryptocurrency prices, answer with a number close to 0. In other words, after reading the news article headline and content, respond with a real number between -1 and 1, indicating whether it's good news, bad news, or uncertain for the cryptocurrency price the next day. If there are multiple articles or contents in the article, consider them comprehensively and respond with only one number instead of multiple numbers. Answer with the number only, without explanation or additional text. Article: {article_text}"

    # 토큰화 및 최대 길이에 맞추어 자르기
    tokens = tokenizer.tokenize(model_input)
    max_length = 1000  # BART 모델의 최대 입력 토큰 길이
    truncated_tokens = tokens[:max_length]

    # 잘린 토큰을 다시 텍스트로 변환
    truncated_text = tokenizer.convert_tokens_to_string(truncated_tokens)

    # BART 모델을 사용하여 예측
    batch = tokenizer(truncated_text, return_tensors="pt")
    generated_ids = model.generate(batch["input_ids"], max_length=1024)
    prediction = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # 예측된 텍스트에서 숫자 추출
    score = 0  # 기본값 설정
    try:
        # 숫자가 포함된 부분을 찾아 점수로 변환
        score = float(prediction.split()[-1])
    except ValueError:
        # 예측된 텍스트에 유효한 숫자가 없는 경우
        pass

    return score


In [7]:
# BART 토크나이저 로드
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

title = test_data['title'][3]
content = test_data['content_full'][3]
article_text = f"Title: {title}. Content: {content}"

model_input = f"Forget all previous instructions. Pretend you are a financial expert with experience in recommending financial assets. If the headline and content of the article are certain to be good news for the price of cryptocurrencies traded the next day, answer with a number close to 1. If it is certain to be bad news, answer with a number close to -1. If it is uncertain or there is insufficient information about the impact on cryptocurrency prices, answer with a number close to 0. In other words, after reading the news article headline and content, respond with a real number between -1 and 1, indicating whether it's good news, bad news, or uncertain for the cryptocurrency price the next day. If there are multiple articles or contents in the article, consider them comprehensively and respond with only one number instead of multiple numbers. Answer with the number only, without explanation or additional text. Article: {article_text}"

# 토큰화 및 최대 길이에 맞추어 자르기
tokens = tokenizer.tokenize(model_input)
max_length = 1000  # BART 모델의 최대 입력 토큰 길이
truncated_tokens = tokens[:max_length]

# 잘린 토큰을 다시 텍스트로 변환
truncated_text = tokenizer.convert_tokens_to_string(truncated_tokens)

# 결과 출력
print("원본 텍스트 길이:", len(tokens))
print("잘린 텍스트 길이:", len(truncated_tokens))
print("잘린 텍스트:", truncated_text)


원본 텍스트 길이: 1749
잘린 텍스트 길이: 1000
잘린 텍스트: Forget all previous instructions. Pretend you are a financial expert with experience in recommending financial assets. If the headline and content of the article are certain to be good news for the price of cryptocurrencies traded the next day, answer with a number close to 1. If it is certain to be bad news, answer with a number close to -1. If it is uncertain or there is insufficient information about the impact on cryptocurrency prices, answer with a number close to 0. In other words, after reading the news article headline and content, respond with a real number between -1 and 1, indicating whether it's good news, bad news, or uncertain for the cryptocurrency price the next day. If there are multiple articles or contents in the article, consider them comprehensively and respond with only one number instead of multiple numbers. Answer with the number only, without explanation or additional text. Article: Title: Bitcoin Surges, Then Sags After 

In [8]:
# 각 기사에 대해 점수 계산
test_data['score'] = test_data.apply(lambda row: calculate_score(row['title'], row['content_full']), axis=1)


<ipython-input-8-a0e1eb757606>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['score'] = test_data.apply(lambda row: calculate_score(row['title'], row['content_full']), axis=1)


In [9]:
test_data

,source,author,title,description,url,urlToImage,time,content,press,content_full,date,score
0,"{'id': 'australian-financial-review', 'name': ...","Ben Potter, AFR.com",This Australian startup will offer shares in l...,Perth-based BitCar is seeking more than $US20 ...,https://www.afr.com/news/bitcar-ico-seeks-25m-...,https://edge.alluremedia.com.au/uploads/busine...,2017-11-09 11:44:01+00:00,Perth-based BitCar is seeking more than $US20 ...,Australian Financial Review,Perth-based BitCar is seeking more than $US20 ...,2017-11-09,0.0
1,"{'id': None, 'name': ""Barron's""}",Avi Salzman and Evie Liu,Bitcoin: Millennials and Baby Boomers Are Split,When it comes to the highly volatile digital a...,http://www.barrons.com/articles/bitcoin-millen...,https://si.wsj.net/public/resources/images/ON-...,2017-11-09 21:45:02+00:00,The generational split is evident in a new sur...,Barron's,"Millennials, who came of age during the Great ...",2017-11-09,0.0
2,"{'id': None, 'name': 'Bitcoinist'}",Antonio Madeira,5 Blockchain-Based Diversification Options for...,Cryptocurrencies are exploding. Bitcoin has re...,http://bitcoinist.com/5-blockchain-based-diver...,http://bitcoinist.com/wp-content/uploads/2017/...,2017-11-09 20:45:10+00:00,"Antonio Madeira · November 9, 2017 · 3:45 pm C...",Bitcoinist,Bitcoin Mining Bitcoin Mining ...,2017-11-09,0.0
3,"{'id': 'breitbart-news', 'name': 'Breitbart Ne...",Ezra Dulis,"Bitcoin Surges, Then Sags After Investors Lose...","The price of Bitcoin surged to nearly $8,000 p...",http://www.breitbart.com/tech/2017/11/09/bitco...,http://media.breitbart.com/media/2017/11/10867...,2017-11-09 18:32:29+00:00,"If you have no idea what any of that means, he...",Breitbart News,"The price of Bitcoin surged to nearly $8,000 p...",2017-11-09,0.0
4,"{'id': 'breitbart-news', 'name': 'Breitbart Ne...",Edwin Mora,Indonesia: Islamic State Primarily Relying on ...,Sympathizers attempting to finance jihadi atta...,http://www.breitbart.com/national-security/201...,http://media.breitbart.com/media/2017/06/finge...,2017-11-09 19:25:08+00:00,A new joint study by Indonesia’s National Coun...,Breitbart News,Sympathizers attempting to finance jihadi atta...,2017-11-09,2016.0
5,"{'id': 'business-insider', 'name': 'Business I...",Elena Holodny,STOCKS DIP: Here's what you need to know,Stocks slid on Thursday amid worries that the ...,http://www.businessinsider.com/stock-market-ne...,http://static6.businessinsider.com/image/5a04b...,2017-11-09 21:00:00+00:00,Stoyan Nenov/Reuters Stocks slid on Thursday a...,Business Insider,Stocks slid on Thursday amid worries that the ...,2017-11-09,0.0
6,"{'id': None, 'name': 'CNA'}",NaN,Nvidia Corp reports 54.6 percent rise in quart...,Nvidia Corp reported a 54.6 percent rise in q...,http://www.channelnewsasia.com/news/business/n...,http://www.channelnewsasia.com/image/9392908/1...,2017-11-09 21:40:14+00:00,REUTERS: Nvidia Corp reported a 54.6 percent r...,CNA,Best News Website or Mobile Service WAN-IFRA...,2017-11-09,0.0
7,"{'id': None, 'name': 'CoinDesk'}",Nikhilesh De,More Charges Filed Against Trader Who Used Bit...,A Philadelphia day trader was formally indicte...,https://www.coindesk.com/day-trader-formally-i...,https://media.coindesk.com/uploads/2017/11/Unt...,2017-11-09 23:00:16+00:00,New charges have been filed against a Philadel...,CoinDesk,New charges have been filed against a Philadel...,2017-11-09,0.0
8,"{'id': None, 'name': 'DailyFX'}","DailyFX Research,, DailyFX Research",Asia AM Digest: Aussie Dollar Volatility Unlik...,The Australian Dollar is unlikely to find fuel...,https://www.dailyfx.com/forex/market_alert/201...,NaN,2017-11-09 23:30:00+00:00,Get the Asia AM Digest every day before Tokyo ...,DailyFX,Get the Asia AM Digest every day before Tokyo ...,2017-11-09,0.0
9,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Overstock shares rocket as sale of home e-comm...,Overstock’s shares shoot up to 12½-year high a...,http://www.marketwatch.com/story/overstock-sha...,http://s.marketwatch.com/publ